In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from urllib.parse import urlencode

import re
#reactor restart
from crochet import setup, wait_for
setup()

API_KEY='b6d2ea20-0a70-40c1-9ca0-0de5564db9ec'

In [2]:
def get_scrapeops_url(url):
    payload={'api_key':API_KEY, 'url':url, 'bypass': 'cloudflare'}
    proxy_url='https://proxy.scrapeops.io/v1/?' + urlencode(payload)
    print(proxy_url)
    return proxy_url

In [4]:
class QuotesSpider(scrapy.Spider):
    name = 'spiderJusBrasil'

    custom_settings={
        'FEEDS':{
            'quotes.csv':{
                'format': 'csv',
                'overwrite': True
            }
        }
    }
    def parse(self, response):
        #quotes = response.xpath('*//div[@class="search-results-subtitle_SearchResultsSubtitle__6_ecG"]/h1[@class="search-results-subtitle_SearchResultsSubtitle-title__3itWy search-results-subtitle_SearchResultsSubtitle-title--marginBottomLarge__2WwRR"]/strong/text()').get()
        #quotes = response.xpath('*//section/h2[@class="search-results_SearchResults-title__QeJBI"]')
        quotes = response.xpath('*//div[@data-testid="search-results"]/section')
       
        for quote in quotes:
            tituloAux2=''
            texto=''.join(quote.xpath('.//h2[@class="search-results_SearchResults-title__QeJBI"]/text()').extract())
            #titulo=quote.xpath('*//a[@data-testid="search-snippet-title-link"]/text()').extract()
            
            #titulosAux =  quote.xpath('*//a[@data-testid="search-snippet-title-link"]/b/text()').getall()

            titulos = quote.xpath('*//a[@data-testid="search-snippet-title-link"]')
            titulo = ''.join([t.xpath('.//text()').extract_first() for t in titulos])

            print(len(titulo))
            print(len(titulosAux))
            tituloAux2=''.join(titulosAux)
            print(tituloAux2)
            tituloFinal = tituloAux2.join(titulo)
            yield{
                'texto': texto,
                'title': titulo
                #'texto': quote.
            }
            

    def start_requests(self):
        urls=['https://www.jusbrasil.com.br/busca?q=seguran%C3%A7a+de+dados']
        
        for url in urls:
            yield scrapy.Request(url=get_scrapeops_url(url), callback=self.parse)

In [5]:
wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesSpider)
    return d

In [7]:
run_spider()

https://proxy.scrapeops.io/v1/?api_key=b6d2ea20-0a70-40c1-9ca0-0de5564db9ec&url=https%3A%2F%2Fwww.jusbrasil.com.br%2Fbusca%3Fq%3Dseguran%25C3%25A7a%2Bde%2Bdados&bypass=cloudflare
147
